In [7]:
import pandas as pd
import numpy as np
import math
df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY - Copy.csv')
df

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200103,9:30:00 AM,SPY,P,320.5,20200103,0.57,0.60,0.47,0.47,321.13,321.15,321.55,321.56,0.5865,241,17
1,20200103,9:30:00 AM,SPY,P,305.0,20200103,0.01,0.01,0.01,0.01,321.13,321.16,321.13,321.16,0.0100,6,1
2,20200103,9:30:00 AM,SPY,P,317.0,20200103,0.12,0.12,0.10,0.10,321.18,321.20,321.33,321.34,0.1018,11,2
3,20200103,9:30:00 AM,SPY,P,313.0,20200103,0.02,0.02,0.02,0.02,321.44,321.45,321.44,321.45,0.0200,1,1
4,20200103,9:30:00 AM,SPY,P,321.5,20200103,0.98,0.98,0.82,0.82,321.14,321.16,321.47,321.48,0.9241,279,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300055,20200311,4:14:00 PM,SPY,C,274.0,20200311,1.06,1.11,1.02,1.05,274.65,274.71,274.62,274.68,1.0830,23,7
300056,20200311,4:14:00 PM,SPY,C,271.0,20200311,3.75,3.75,3.75,3.75,274.67,274.71,274.67,274.71,3.7500,1,1
300057,20200311,4:14:00 PM,SPY,C,276.0,20200311,0.27,0.28,0.17,0.25,274.70,274.74,274.59,274.60,0.2406,17,8
300058,20200311,4:14:00 PM,SPY,C,282.0,20200311,0.01,0.01,0.01,0.01,274.54,274.59,274.54,274.59,0.0100,3,2


## Get minimum value for call/put between 9:46AM and the timecallmax for that day

In [10]:
#function that compares if time 1 is < time 2
def beforeTime(time1,time2):
    
    if len(time1) != 11:
        time1 = "0" + time1
    if len(time2) != 11:
        time2 = "0" + time2
    
    #check last 2 chars to see if AM or PM
    if time1[-2:] == "AM":
        t1IsAM = True
    else:
        t1IsAM = False
    if time2[-2:] == "AM":
        t2IsAM = True
    else:
        t2IsAM = False
    
    if t1IsAM and not t2IsAM:
        return True
    elif t1IsAM and t2IsAM:
        return True if time1 < time2 else False
    elif not t1IsAM and t2IsAM:
        return False
    else: #t1 is pm and t2 is pm
        return True if time1 < time2 else False

#sample test cases - expected TFFF
print(beforeTime("10:05:00 AM","3:59:00 PM"), beforeTime("10:05:00 PM","3:59:00 PM"), beforeTime("10:05:00 AM","3:59:00 AM"),beforeTime("10:05:00 PM","3:59:00 AM"))

True False False False


### Starting price difference - green (increase) and red (decrease) indicators for the first 5 mins of each date

In [29]:
dateTimeCandleSP = [] #(date,time,green/red,Strike Price)
date = df.iloc[0]["Date"]
openPrice = df.iloc[0]["UnderOpenAskPrice"]
skipDate = False
#indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
for index,row in df.iterrows():
    #we are looking for the UnderOpenAskPrice(UOAP) at 9:30:00 AM (the first time it occurs) 
    #we are looking for the UnderOpenAskPrice (UOAP) at 9:36:00 AM (the last time it occurs)
    #then we subtract the 9:35:00 AM time by the 9:30:00 AM time
    
    if date == row[0]:
        if skipDate:
            continue
        #lastPrice = row["UnderCloseAskPrice"]
        prevRow = row
        #if time is past 9:35AM, then use the latest 9:35AM (if it doesn't exist, then use the current row's time)
        time = row[1]
        if not beforeTime(time,"9:36:00 AM"):
            #calculate starting price difference
            
            #if 9:35 exists
            if prevRow[1] == "9:35:00 AM":
                #we take the UnderCloseAskPrice
                lastPrice = prevRow[13]
            else:
                #if 9:35 doesn't exist
                #if 9:36 exists, then take the open price, otherwise skip this date
                if time == "9:36:00 AM":
                    lastPrice = row[11]
                else:
                    skipDate = True
                    continue
            diff = lastPrice - openPrice
            if math.isnan(diff):
                #either openPrice or lastPrice doesn't exist
                skipDate = True
                continue
            #print("date={},lastPrice={}".format(date,lastPrice))
            if diff >= 0:
                #green start
                dateTimeCandleSP.append((date,time,"GREEN",math.ceil(lastPrice),lastPrice))
            else:
                dateTimeCandleSP.append((date,time,"RED",math.floor(lastPrice),lastPrice))
            skipDate = True
    else:
        #we found new date, get the green/red bar for the previous date, reinitialize some variables
        date = row[0]
        openPrice = row[11]
        skipDate = False

In [30]:
dateTimeCandleSP

[(20200103, '9:36:00 AM', 'GREEN', 323, 322.41),
 (20200106, '9:36:00 AM', 'GREEN', 322, 321.24),
 (20200108, '9:36:00 AM', 'GREEN', 324, 323.4),
 (20200110, '9:36:00 AM', 'RED', 326, 326.84),
 (20200113, '9:36:00 AM', 'RED', 326, 326.18),
 (20200115, '9:36:00 AM', 'GREEN', 329, 328.08),
 (20200117, '9:36:00 AM', 'RED', 331, 331.45),
 (20200121, '9:36:00 AM', 'GREEN', 332, 331.4),
 (20200122, '9:36:00 AM', 'GREEN', 333, 332.44),
 (20200124, '9:36:00 AM', 'RED', 332, 332.17),
 (20200127, '9:36:00 AM', 'GREEN', 324, 323.77),
 (20200129, '9:36:00 AM', 'RED', 328, 328.41),
 (20200131, '9:36:00 AM', 'RED', 326, 326.16),
 (20200203, '9:36:00 AM', 'GREEN', 325, 324.16),
 (20200205, '9:36:00 AM', 'RED', 331, 331.93),
 (20200207, '9:36:00 AM', 'RED', 332, 332.26),
 (20200210, '9:36:00 AM', 'GREEN', 333, 332.14),
 (20200212, '9:36:00 AM', 'GREEN', 338, 337.03),
 (20200214, '9:36:00 AM', 'GREEN', 338, 337.51),
 (20200218, '9:36:00 AM', 'GREEN', 337, 337.0),
 (20200219, '9:36:00 AM', 'GREEN', 338,

In [ ]:
for date,time,color in dateTimeCandleSP:
    if color == "GREEN":
        if time == "9:35:00 AM":
            call_SP =

dtcSPidx = 0
for index,row in df.iterrows():
    


#if the candle is green then we want to buy a call
#if call then you look for the UnderOpenAskPrice at 9:36:00 AM and round that up to next whole number, this gets the Call_SP
#then we look for the Call_SP and see what the OpenTradePrice is at 9:36:00 AM


#if the candle is red then we want to buy a put
#if put then you look for the UnderOpenAskPrice at 9:36:00 AM and round that down to next whole number, this gets the Put_SP
#then we look for the Put_SP and see what the OpenTradePrice is at 9:36:00 AM
